In [ ]:
import pandas as pd
from thefuzz import fuzz
from thefuzz import process
import googlemaps
import geopandas as gpd
import matplotlib.pyplot as plt
import urllib.request, json 
with urllib.request.urlopen('https://www.abgeordnetenwatch.de/api/v2/candidacies-mandates?parliament_period=132&range_end=750') as url:
    can_man = json.load(url) # used to assign voting districts to member of the Bundestag


In [ ]:
#!pip install thefuzz
#!pip install levenshtein
#!pip install -U googlemaps
#!pip install geopandas
#!pip install folium matplotlib mapclassify


In [ ]:
#
# The first part establishes a DataFrame with the name, email adress and voting districts of all members of parliament
#

In [ ]:
# Import the document "Kürschners Volkshandbuch", which provides names and E-Mail Adresses
# The document can be downloaded here: https://www.btg-bestellservice.de/pdf/10037700.pdf; it was converted to a txt file
myfile = open("Liste_MdBs.txt", "rt")
file = myfile.read()
# Split the huge string into a list of strings at each new line
txt_into_list = file.split("\n")
# extract all E-Mail adresses that end with @bundestag.de
result = [x for x in txt_into_list if x.startswith('E-Mail:') and x.endswith('@bundestag.de')]
mail_adress = [x.removeprefix('E-Mail: ') for x in result]
# reconstruct names by splitting them up at each dot before the @ and combining them again
names = [x.removesuffix('@bundestag.de') for x in mail_adress]
df_mail = pd.DataFrame(list(zip(names, mail_adress)),
                       columns= ['name_conc', 'mail_adress'])
df_mail[['Vorname', 'Nachname']] = df_mail.name_conc.str.split('.', expand=True)
df_mail["Name"] = df_mail['Vorname'] + " " + df_mail['Nachname']
# remove the 2 E-Mail adresses that don't correspond to members of parliament
df_mail = df_mail.dropna(subset=['Name']).reset_index(drop=True)
df_mail.to_csv('Output\Liste_MdB_Mail.csv')


In [ ]:
# Import the list of member of parliament. The list was copied from: https://www.bundestag.de/abgeordnete; You'd need to switch into list view before copying the list.
# Importantly the stars that would assign the reasons for why the member of parliament is no longer a member of parliament, were replaced by the character x.
df_np = pd.read_csv('Liste_MdB_Namen.csv', header=None)
# The party affiliations are stored in even row numbers
df_party = df_np[df_np.index % 2 != 0].reset_index(drop=True)
df_party.columns = ['Partei', 'junk']
#The names of members of parliament are stored in odd row numbers in 2 columns
df_name = df_np[df_np.index % 2 == 0].reset_index(drop=True)
df_name.columns = ['Nachname', 'Vorname']
# combine the names and party affiliations
df_name_party = df_name.join(df_party)
# remove leading whitespaces from names and party affiliations
df_name_party['Nachname'] = df_name_party.Nachname.str.lstrip()
df_name_party['Vorname'] = df_name_party.Vorname.str.lstrip()
df_name_party['Partei'] = df_name_party.Partei.str.lstrip()
# remove everything in parentheses. Parentheses were used when names occured more than once and provided information of the home district of the member of parliament
df_name_party['Nachname'] = df_name_party['Nachname'].str.replace(r"\(.*\)","")
# Reconstruct Names by combining names and surnames; export the DataFrame
df_name_party['Name'] = df_name_party['Vorname'] + " " + df_name_party['Nachname']
df_name_party.to_csv('Output\Liste_MdB_Name_Partei.csv')

In [ ]:
# In order to combine the mail DataFrame and the name DataFrame, a common column for joining the DataFrames would be needed.
# fuzzy search was applied to connect the reconstructed names from the mail DataFrame with the true names of the name DataFrame
# Setting the right algorithm for the fuzzy search is crucial. token_set_ratio provided the best result, even when academic titles were involved.
df_mail = pd.read_csv('Output\Liste_MdB_Mail.csv')
df_name_party = pd.read_csv('Output\Liste_MdB_Name_Partei.csv')
df_mail['Name_fuzzy'] = df_mail['Name'].apply(
  lambda x: process.extractOne(x, df_name_party['Name'], scorer=fuzz.token_set_ratio)[0]
)

In [ ]:
# This part is optional. The scores of the fuzzy search are calculated and added to the DataFrame df_mail.
ratios = []
for i in df_mail.index:
    ratios.append(fuzz.token_sort_ratio(df_mail['Name'][i], df_mail['Name_fuzzy'][i]))
df_mail['Score'] = ratios
# Check if there were mismatches. If there were mismatches of the reconstructed and true names, one would expect duplicated entries inm the Name_fuzzy column.
df_mail[df_mail.duplicated(['Name_fuzzy'], keep=False)].sort_values(by=['Name_fuzzy', 'Score'])
# Export df_mail as csv
df_mail.to_csv('Output\Liste_MdB_Mail_Scores.csv')

In [ ]:
# create new DataFrame from the mail DataFrame that only includes the E-mail adresses and the matched names from df_name_party. Rename column Name_fuzzy to Name
df_mail_strip = df_mail[['Name_fuzzy', 'mail_adress']].copy()
df_mail_strip.columns = ['Name', 'mail_adress']
# merge the name_party and mail DataFrames on the column Name
data_name_party_adress = pd.merge(df_name_party, df_mail_strip, on='Name', how="left")
# remove all previous members of parliament, by removing all entries that contain the character x in the Partei column 
data_name_party_adress = data_name_party_adress[~data_name_party_adress['Partei'].str.contains('x')].reset_index(drop=1)

In [ ]:
# check for entries that lack E-Mail adresses
data_name_party_adress[data_name_party_adress['mail_adress'].isna()]

In [ ]:
# grab indices of entries without E-mail adresses and add the missing adresses by hand
na_ind = data_name_party_adress[data_name_party_adress['mail_adress'].isna()].index
data_name_party_adress.loc[na_ind, 'mail_adress'] = ['melanie.bernstein@bundestag.de', 'alexander.foehr@bundestag.de', 'dirk-ulrich.mende@bundestag.de', 'Rainer.Rothfuss@bundestag.de', 'ana-maria-trasnea@bundestag.de.', 'emily.vontz@bundestag.de']
# Export the DataFrame
data_name_party_adress.to_csv('Output\Liste_MdB_Name_Partei_Adresse.csv')

In [ ]:
# Use the json file from the API of abgeordnetenwatch.de to assign voting districts to the members of the Bundestag
# extract the name of a politician
can_man.get('data')[250].get('politician').get('label')
# extract the voting district
can_man.get('data')[250].get('electoral_data').get('constituency').get('label')
# create 2 lists with names and coresponding voting districts
length = len(can_man.get('data'))
name = []
wahlkreis = []
for i in range(length):
    name.append(can_man.get('data')[i].get('politician').get('label'))
    if can_man.get('data')[i].get('electoral_data').get('constituency') == None:
        wahlkreis.append('None')
    else:
         wahlkreis.append(can_man.get('data')[i].get('electoral_data').get('constituency').get('label'))

# combine the 2 lists to one DataFrame
mdb_wkr = pd.DataFrame(list(zip(name, wahlkreis)), columns=['Name', 'Wahlkreis'])
# remove everything in parentheses (Bundestag 2021-2025)
mdb_wkr['Wahlkreis'] = mdb_wkr['Wahlkreis'].str.replace(r"\(.*\)","")
# Split the number of the voting district and its name
mdb_wkr[['Wahlkreis', 'Wahlkreisname']] = mdb_wkr.Wahlkreis.str.split(" - ", 1, expand=True)
# create a common column with df_name_party and data_name_party_adress
mdb_wkr['Name_fuzzy'] = mdb_wkr['Name'].apply(
  lambda x: process.extractOne(x, df_name_party['Name'], scorer=fuzz.token_set_ratio)[0]
)

In [ ]:
# This part is optional. The scores of the fuzzy search are calculated and added to the DataFrame mdb_wkr.
ratios = []
for i in mdb_wkr.index:
    ratios.append(fuzz.token_sort_ratio(mdb_wkr['Name'][i], mdb_wkr['Name_fuzzy'][i]))
mdb_wkr['Score'] = ratios
# Check if there were mismatches. If there were mismatches of the reconstructed and true names, one would expect duplicated entries inm the Name_fuzzy column.
mdb_wkr[mdb_wkr.duplicated(['Name_fuzzy'], keep=False)].sort_values(by=['Name_fuzzy', 'Score'])
mdb_wkr.to_csv('Output\Liste_MdB_Wahlkreis.csv')

In [ ]:
# create new DataFrame from the mail DataFrame that only includes the E-mail adresses and the matched names from df_name_party. Rename column Name_fuzzy to Name
mdb_wkr_strip = mdb_wkr[['Name_fuzzy', 'Wahlkreis', 'Wahlkreisname']].copy()
mdb_wkr_strip.columns = ['Name', 'Wahlkreis', 'Wahlkreisname']
# merge the name_party and mail DataFrames on the column Name
data_name_party_adress_wkr = pd.merge(data_name_party_adress, mdb_wkr_strip, on='Name', how="left")
# convert the column to numeric and replace 'None' with nan
data_name_party_adress_wkr['Wahlkreis'] = pd.to_numeric(data_name_party_adress_wkr['Wahlkreis'], errors='coerce')
# Export the DataFrame
data_name_party_adress_wkr.to_csv('Output\Liste_MdB_Name_Partei_Adresse_Wahlkreis.csv')

In [ ]:
#
# The second part establishes a DataFrame with all universities and their corresponding voting districts
#

In [ ]:
# Do this only if you have a valid Google Maps API key. If you don't have one skip this cell and proceed with the next one. You'll find a hs_liste_koordinaten.csv file with all coordinates in the output folder. 

api = open("Google maps api.txt", "rt")
api_str = api.read()
# Add your individual Google Maps API key
gmaps = googlemaps.Client(key=api_str)

# import DataFrame with a list of all universities in Germany. Concatenate the adressess
df_hochschulen = pd.read_csv('hs_liste.txt', sep='\t', dtype=str)
adress = df_hochschulen['Straße'] + ', ' + df_hochschulen['Postleitzahl (Hausanschrift)'] + ', ' + df_hochschulen['Ort (Hausanschrift)']
df_hochschulen['Adresse'] = adress

# find the coordinates for all university adresses. Store the values in separate lists
lattitude = []
longitude = []
# Do this only once since it grabs the information via the googlemaps API which isn't technically free
for i in df_hochschulen.index:
    x = gmaps.geocode(df_hochschulen['Adresse'][i])
    lattitude.append(x[0].get('geometry').get('location')['lat'])
    longitude.append(x[0].get('geometry').get('location')['lng'])
# add the coordinates to the original DataFrame and save it.
df_hochschulen['Latitude'] = lattitude
df_hochschulen['Longitude'] = longitude
df_hochschulen.to_csv('Output\hs_liste_koordinaten.csv', sep='\t')

In [ ]:
# Import the shp files of all voting districts in Germany and change their reference system to degrees
# download the zip file from: https://www.bundeswahlleiterin.de/dam/jcr/fc2f7796-cc84-4f89-b475-4439fcc8fa07/btw21_geometrie_wahlkreise_shp.zip and extract to \Wahlkreise
# You can find all kind of data regarding voting districts here: https://www.bundeswahlleiterin.de/bundestagswahlen/2021/wahlkreiseinteilung/downloads.html
Wahlkreise = gpd.read_file('Wahlkreise\Geometrie_Wahlkreise_20DBT.shp')
Wahlkreise_Grad = Wahlkreise.to_crs("EPSG:4326")

In [ ]:
# make df_hochschulen DataFrame a geopanda GeoDataFrame by specifying the columns with longitude and latitude and specifying the degree reference system
df_hochschulen = pd.read_csv('Output\hs_liste_koordinaten.csv', sep='\t') # load hs_liste_koordinaten.csv in case you don't have a Google Maps API
df_hochschulen_shp = gpd.GeoDataFrame(
    df_hochschulen, geometry=gpd.points_from_xy(df_hochschulen.Longitude, df_hochschulen.Latitude), crs="EPSG:4326"
)

In [ ]:
# Optional: plot all universities onto the borders of all voting districts
ax = Wahlkreise_Grad.plot(color="white", edgecolor="black")
df_hochschulen_shp.plot(ax = ax, color = 'red', markersize = 2)

In [ ]:
# assign the voting districts to the universities
df_hochschulen_wkr = gpd.tools.sjoin(df_hochschulen_shp, Wahlkreise_Grad, predicate = 'within', how = 'left')
# export the DataFrame
df_hochschulen_wkr.to_csv('Output\hs_liste_koordinaten_Wahlkreise.csv', sep='\t')

In [ ]:
#
# The third part combines universities and member of parliament
#

In [ ]:
# Create a DataFrame that combines the names, Email-Adresses and the names of universities
list_hs_kurz = []
list_hs_lang = []
for i in data_name_party_adress_wkr.index:
    list_hs_kurz.append(df_hochschulen_wkr[(df_hochschulen_wkr.WKR_NR == data_name_party_adress_wkr['Wahlkreis'][i])]['Hochschulkurzname'].tolist())
    list_hs_lang.append(df_hochschulen_wkr[(df_hochschulen_wkr.WKR_NR == data_name_party_adress_wkr['Wahlkreis'][i])]['Hochschulname'].tolist())

data_name_party_adress_wkr_hs = data_name_party_adress_wkr
data_name_party_adress_wkr_hs['HS_Kurzname'] = list_hs_kurz
data_name_party_adress_wkr_hs['HS_Name'] = list_hs_lang

In [ ]:
# remove all MdBs that don't represent a district with at least 1 university
data_name_party_adress_wkr_hs = data_name_party_adress_wkr_hs[data_name_party_adress_wkr_hs['HS_Name'].map(lambda d: len(d)) > 0]
# drop unimportant columns
data_name_party_adress_wkr_hs.drop(['Unnamed: 0', 'Nachname', 'Vorname', 'junk', 'Wahlkreisname'], axis=1, inplace=True)
data_name_party_adress_wkr_hs.to_csv('Output\Liste_MdB_Name_Partei_Adresse_Wahlkreis_Hochschule.csv', sep='\t')

In [ ]:
# Create a DataFrame that combines, universities, Names and Email-Adresses of the MdBs

#first create empty lists for the names and adresses for the invdividual parties
SPD_Name = []
SPD_Adr = []
CDU_Name = []
CDU_Adr = []
Green_Name = []
Green_Adr = []
FDP_Name = []
FDP_Adr = []
Linke_Name = []
Linke_Adr = []
AFD_Name = []
AFD_Adr = []

# iterate through each row of df_hochschulen_wkr and look in data_name_party_adress_wkr whether there is an MdB of the respective party. If not append 'none' to the lists, if yes append the Name and Email Adresses to the respective lists
for i in df_hochschulen_wkr.index:
    if len(data_name_party_adress_wkr[(data_name_party_adress_wkr.Wahlkreis == df_hochschulen_wkr.WKR_NR[i]) & (data_name_party_adress_wkr.Partei == 'SPD')]['Name']) == 1:
        SPD_Name.append(data_name_party_adress_wkr[(data_name_party_adress_wkr.Wahlkreis == df_hochschulen_wkr.WKR_NR[i]) & (data_name_party_adress_wkr.Partei == 'SPD')]['Name'].reset_index(drop=True)[0])
        SPD_Adr.append(data_name_party_adress_wkr[(data_name_party_adress_wkr.Wahlkreis == df_hochschulen_wkr.WKR_NR[i]) & (data_name_party_adress_wkr.Partei == 'SPD')]['mail_adress'].reset_index(drop=True)[0])
    else:
        SPD_Name.append('None')
        SPD_Adr.append('None')
    
    if len(data_name_party_adress_wkr[(data_name_party_adress_wkr.Wahlkreis == df_hochschulen_wkr.WKR_NR[i]) & (data_name_party_adress_wkr.Partei == 'CDU/CSU')]['Name']) == 1:
        CDU_Name.append(data_name_party_adress_wkr[(data_name_party_adress_wkr.Wahlkreis == df_hochschulen_wkr.WKR_NR[i]) & (data_name_party_adress_wkr.Partei == 'CDU/CSU')]['Name'].reset_index(drop=True)[0])
        CDU_Adr.append(data_name_party_adress_wkr[(data_name_party_adress_wkr.Wahlkreis == df_hochschulen_wkr.WKR_NR[i]) & (data_name_party_adress_wkr.Partei == 'CDU/CSU')]['mail_adress'].reset_index(drop=True)[0])
    else:
        CDU_Name.append('None')
        CDU_Adr.append('None')

    if len(data_name_party_adress_wkr[(data_name_party_adress_wkr.Wahlkreis == df_hochschulen_wkr.WKR_NR[i]) & (data_name_party_adress_wkr.Partei == 'Bündnis 90/Die Grünen')]['Name']) == 1:
        Green_Name.append(data_name_party_adress_wkr[(data_name_party_adress_wkr.Wahlkreis == df_hochschulen_wkr.WKR_NR[i]) & (data_name_party_adress_wkr.Partei == 'Bündnis 90/Die Grünen')]['Name'].reset_index(drop=True)[0])
        Green_Adr.append(data_name_party_adress_wkr[(data_name_party_adress_wkr.Wahlkreis == df_hochschulen_wkr.WKR_NR[i]) & (data_name_party_adress_wkr.Partei == 'Bündnis 90/Die Grünen')]['mail_adress'].reset_index(drop=True)[0])
    else:
        Green_Name.append('None')
        Green_Adr.append('None')

    if len(data_name_party_adress_wkr[(data_name_party_adress_wkr.Wahlkreis == df_hochschulen_wkr.WKR_NR[i]) & (data_name_party_adress_wkr.Partei == 'FDP')]['Name']) == 1:
        FDP_Name.append(data_name_party_adress_wkr[(data_name_party_adress_wkr.Wahlkreis == df_hochschulen_wkr.WKR_NR[i]) & (data_name_party_adress_wkr.Partei == 'FDP')]['Name'].reset_index(drop=True)[0])
        FDP_Adr.append(data_name_party_adress_wkr[(data_name_party_adress_wkr.Wahlkreis == df_hochschulen_wkr.WKR_NR[i]) & (data_name_party_adress_wkr.Partei == 'FDP')]['mail_adress'].reset_index(drop=True)[0])
    else:
        FDP_Name.append('None')
        FDP_Adr.append('None')

    if len(data_name_party_adress_wkr[(data_name_party_adress_wkr.Wahlkreis == df_hochschulen_wkr.WKR_NR[i]) & (data_name_party_adress_wkr.Partei == 'Die Linke')]['Name']) == 1:
        Linke_Name.append(data_name_party_adress_wkr[(data_name_party_adress_wkr.Wahlkreis == df_hochschulen_wkr.WKR_NR[i]) & (data_name_party_adress_wkr.Partei == 'Die Linke')]['Name'].reset_index(drop=True)[0])
        Linke_Adr.append(data_name_party_adress_wkr[(data_name_party_adress_wkr.Wahlkreis == df_hochschulen_wkr.WKR_NR[i]) & (data_name_party_adress_wkr.Partei == 'Die Linke')]['mail_adress'].reset_index(drop=True)[0])
    else:
        Linke_Name.append('None')
        Linke_Adr.append('None')

    if len(data_name_party_adress_wkr[(data_name_party_adress_wkr.Wahlkreis == df_hochschulen_wkr.WKR_NR[i]) & (data_name_party_adress_wkr.Partei == 'AfD')]['Name']) == 1:
        AFD_Name.append(data_name_party_adress_wkr[(data_name_party_adress_wkr.Wahlkreis == df_hochschulen_wkr.WKR_NR[i]) & (data_name_party_adress_wkr.Partei == 'AfD')]['Name'].reset_index(drop=True)[0])
        AFD_Adr.append(data_name_party_adress_wkr[(data_name_party_adress_wkr.Wahlkreis == df_hochschulen_wkr.WKR_NR[i]) & (data_name_party_adress_wkr.Partei == 'AfD')]['mail_adress'].reset_index(drop=True)[0])
    else:
        AFD_Name.append('None')
        AFD_Adr.append('None')

#Create a temporary Dataframe with the filled lists and join it with df_hochschulen_wkr
temp = pd.DataFrame(list(zip(SPD_Name, SPD_Adr, CDU_Name, CDU_Adr, Green_Name, Green_Adr, FDP_Name, FDP_Adr, Linke_Name, Linke_Adr, AFD_Name, AFD_Adr)), 
                    columns=['SPD_Name', 'SPD_Adr', 'CDU_Name', 'CDU_Adr', 'Green_Name', 'Green_Adr', 'FDP_Name', 'FDP_Adr', 'Linke_Name', 'Linke_Adr', 'AFD_Name', 'AFD_Adr']
)

df_hochschulen_wkr_mdb = pd.concat([df_hochschulen_wkr, temp], axis = 1, join = 'inner')


In [ ]:
# Create a new DataFrame with only important columns
df_hochschulen_wkr_mdb_strip = df_hochschulen_wkr_mdb[['Hochschulkurzname', 'Hochschulname', 'WKR_NR', 'SPD_Name', 'SPD_Adr', 'CDU_Name', 'CDU_Adr', 'Green_Name', 'Green_Adr', 'FDP_Name', 'FDP_Adr', 'Linke_Name', 'Linke_Adr', 'AFD_Name', 'AFD_Adr', 'Hochschultyp', 'Trägerschaft', 'Bundesland']].copy()
df_hochschulen_wkr_mdb_strip.to_csv('Output\hs_liste_Wahlkreise_mdb.csv', sep='\t')